# Assignment 4: Logistic Regression and Gaussian Process Regression

### Machine Learning Basic Module
Florian Walter, Tobias Jülg, Pierre Krack

## General Information About Implementation Assignments
We will use the Jupyter Notebook for our implementation exercises. The task description will be provided in the notebook. The code is also run in the notebook. However, the implementation itself is done in additional files which are imported in the notebook. Please do not provide any implementation that you want to be considered for correction in this notebook, but only in Python files in the marked positions. A content of a Python file could for example look similar as shown below:
```python
def f():
    ########################################################################
    # YOUR CODE
    # TODO: Implement this function
    ########################################################################
    pass
    ########################################################################
    # END OF YOUR CODE
    ########################################################################
```
To complete the exercise, remove the `pass` command and only use space inside the `YOUR CODE` block to provide a solution. Other lines within the file may not be changed in order to deliver a valid submission.


### Imports

In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
from urllib.request import urlretrieve

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from gpr import load_and_split, train_and_predict

width = 9.5
plt.rcParams['figure.figsize'] = [width, width / 1.618] 
plt.rcParams['figure.dpi'] = 100
UTNRED = "#f5735f"
UTNBLUE = "#0087dc"
mpl.rcParams['path.simplify'] = True

## Gaussian Process Regression

This exercise will focus more on the practical ML workflow: you will use a real recent robotics dataset to learn the forward dynamics of a robot.
The result will be a predictor that can be used to simulate the movement of this robot, which could be used to generate more data or for reinforcement learning purposes.

The goal will be to use the [sklearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html) of Gaussian Process Regression to learn the forward dynamics of the 3 DoF robot arm shown below.

<img src="https://raw.githubusercontent.com/rr-learning/transferable_dynamics_dataset/master/img/16.png" width=500 height=500 />

Because the focus is on a real-world workflow and not on understanding some theoretical concept, this exercise will be less guided than the ones before.

While working through the assignment, you will probably have many questions.
Please post them on Canvas.
We will look at this discussion to make sure you don't get stuck and are not going down a wrong path.

### The dataset
Every machine learning project (except for reinforcement learning) starts with a dataset.
Sometimes that dataset is given and your task is to analyze it (data science), other times the dataset is a means to an end.
In the latter case, the first step involves searching for datasets that are suited to your task, potentially comparing a few, or, in the worst case, creating a new dataset—a potentially massive undertaking.

For the purpose of this exercise, we did this first step for you!
Visit the dataset's [github repository](https://github.com/rr-learning/transferable_dynamics_dataset/tree/master).
The [corresponding paper](https://ieeexplore.ieee.org/abstract/document/9197392) contains additional information that is relevant for this exercise.
You should be able to access this paper through UTN if you are in the eduroam network at UTN.
Let us know if you can not access it for some reason.
This dataset was first published at the [2020 International Conference on Robotics and Automation (ICRA)](https://ieeexplore.ieee.org/xpl/conhome/9187508/proceeding), a conference you will often encounter during your studies, and, if you follow an academic path in the field of robotics, probably visit one day.

We provide our own link to the data; make sure you are connected to the internet (or the server on which this notebook's kernel is running on) and run the next cell to download the dataset.

In [ ]:
FILE_NAME="Sines_full.npz"
DATA_URL=f"https://faubox.rrze.uni-erlangen.de/dl/fiPEC8zDDyJ6sEr9GtBBr4/{FILE_NAME}"
file_path = Path(FILE_NAME)
if not file_path.exists():
    print("Downloading data. Might take a while depending on your internet connection.")
    urlretrieve(DATA_URL, filename=FILE_NAME)
    print("Done")

After you have found a dataset, you will want to use it in a programming language.
Depending on the quality of the dataset, the format it is distributed in and the programming language you use, this can be more or less straightforward.

We, of course, chose a very high quality dataset. 😉

But you still need to understand it.
So this will be your first task!

>**Task 5** Use the website of the dataset, the corresponding publication, and python to load, inspect, understand and finally split the dataset into a training set and a testing set.
Drop some samples from the data such that it looks like it was generated by a 100Hz control loop instead of a 1kHz one (vanilla GPR slows down with large datasets).
Use only the first rollout from the dataset, which corresponds to the lower frequency sine waves and the restricted workspace (i.e. the simplest case).
Use the first five seconds for the training split and the remaining ones for the test split.
Only use the data from the first joint.
>
>Hints:
> * We download only the full dataset, not the pre-split training and testing ones.
This is because we use our own split, since we use vanilla GPR, whereas in the paper they use an extension to GPRs which allows it to learn on larger datasets.
> * The following cell should print: `((500, 1), (500, 1), (1000, 1), (1000, 1))`
> * Don't overdo it, you can solve this task with about five lines. The task is more about researching and understanding than coding.
> * Remember that we want to learn the *forward dynamics* of the robot. What should be X and what should be y?

In [ ]:
x_train, y_train, x_test, y_test = load_and_split(file_path)
tuple(split.shape for split in (x_train, y_train, x_test, y_test))

### Applying GPR
The next part of a typical machine learning project is then to apply machine learning algorithms.
>**Task 6** use the [sklearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html) of GPR with a radial basis function (RBF) kernel to train on the training data, then return the mean and standard deviation of the prediction.
Run the next cell, the plot should resemble this:
![expected result](./expected.png)
You can use the interactive matplotlib interface to zoom in and scroll around.
>
>Hints:
>* You will need to tweak the parameters of the `GaussianProcessRegressor` class.
>* If your results look strange, make sure to look at the sklearn documentation for both the [RBF kernel](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html) and the [`GaussianProcessRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html). It includes examples of how they are used.

In [ ]:
# change to %matplotlib inline if interactive plotting does not work.
%matplotlib notebook 
mean_prediction, std_prediction = train_and_predict(file_path)
fig, axs = plt.subplots(2, sharex=True)
x = np.arange(0, x_test.shape[0]) / 100
axs[0].plot(x, x_test.flatten(), label="desired torques", color=UTNBLUE)
axs[1].plot(x, y_test.flatten(), label="measured angle", color=UTNBLUE)
axs[1].plot(x, mean_prediction, color=UTNRED, label="predicted angle")
axs[1].fill_between(x, mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction, alpha=0.5, color=UTNRED, label="Prediction 95% confidence")
axs[1].set_xlabel("Time (s)")
axs[0].set_ylabel("Torque (Nm)")
axs[1].set_ylabel("Angles (rad)")
axs[0].legend(loc="upper center", bbox_to_anchor=(0.5, 1.2))
axs[1].legend(loc="upper center", bbox_to_anchor=(0.5, 1.2), ncols=3)
plt.show()

The next step would be to analyze the results.
You might run a hyperparameter optimization algorithm, compare the influence of different parameters, measure the errors in your predictions etc. etc.

But you are done for today!